In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions


In [55]:
folder = ""
raw = folder 
raw_dataset_x = pd.read_csv(raw + 'X_train.csv',index_col=['id'])


In [56]:
raw_dataset_y = pd.read_csv(raw + 'y_train.csv',index_col=['id'])
#y = raw_dataset_y.copy()
raw_dataset_y.head()

,y
id,
0,0
1,0
2,0
3,1
4,2


In [57]:
just_scoring = True
if just_scoring:
    preprocessed = folder + "preprocessed_data/"
    x_train, x_test, y_train, y_test = train_test_split(raw_dataset_x, raw_dataset_y, test_size=0.2, random_state=100)
else:
    preprocessed = folder + "ready_data/"
    x_train = pd.read_csv(raw + 'X_train.csv', index_col=['id'])
    x_test = pd.read_csv(raw + 'X_test.csv', index_col=['id'])
    y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
    y_test = pd.DataFrame({'A': []})

In [58]:
#get the row with more missing values
#x_train.isnull().sum(axis=1).sort_values(ascending=False).head(1)

In [59]:
# remove nan from np array preserving dimension
def remove_nan(x):
    return x[~np.isnan(x)]
x_np = np.array([remove_nan(x) for x in x_train.values], dtype=object)


In [60]:
from multiprocessing import Pool
from datetime import time
import functions


def apply_func(row):
    row = pd.Series(row)
    ecg_signal = row.dropna().to_numpy()
    ecg_cleaned = nk.ecg.ecg_clean(ecg_signal, sampling_rate=300)
#     ecg_cleaned = ecg_signal

    max_abs = max(ecg_cleaned.min(), ecg_cleaned.max(), key=abs)
    ecg_cleaned = ecg_cleaned / max_abs

#     # Extract R-peaks locations
    peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=300)


    # Delineate the ECG signal
    if len(rpeaks['ECG_R_Peaks']) > 3:
        signal_dwt, waves_dwt = nk.ecg_delineate(ecg_cleaned, rpeaks, sampling_rate=300, method="dwt")

        for key, arr in waves_dwt.items():
            if 'Onsets' in key or 'Offsets' in key:
                row = add_basic_info(row, np.array(arr), key)
                amplitudes = np.array([ecg_cleaned[i] for i in arr if not np.isnan(i)])
                row = add_basic_info(row, amplitudes, f"{key}_amplitudes")

    return row

#x_train.apply(apply_func,axis = 1)

In [61]:
import scipy
import math
import operator
from functions import *
import itertools

id_error = []

def extrack_features(row,index):
    signal = row.dropna().to_numpy(dtype='float32')
    if index in [1764, 434, 3506, 1713, 4587, 5034]:
        signal = -signal 
    ecg_cleaned = nk.ecg_clean(signal, sampling_rate=300)
    _, r_peaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=300)
    #r_peaks = ecg.engzee_segmenter(signal, 300)['rpeaks']
    r_peaks = r_peaks['ECG_R_Peaks']
    #print(r_peaks)
    if len(r_peaks) >= 2:
        #print(ecg.extract_heartbeats(signal, r_peaks, 300))

        beats = ecg.extract_heartbeats(signal, r_peaks, 300)['templates']

        if len(beats) != 0:
            mu = np.mean(beats, axis=0)
        else:
            mu = np.zeros(180)
    else:
        mu = np.zeros(180)

    try:
        if len(r_peaks)> 3:
            signal_dwt, waves_dwt = nk.ecg_delineate(ecg_cleaned,
                                    r_peaks,
                                    sampling_rate=300,
                                    method="dwt",
                                    show=False,
                                    show_type='all')
            return get_nk_features(signal, r_peaks, waves_dwt)

        else:
            return np.empty(14)
    except Exception as e:
        print(e," - ",index," error")
        id_error.append(index)
        return np.empty(14)


#classify ecg with biosppy
def classify_ecg(x_train):
    x_train_ecg = x_train.copy()
    features = np.empty((0,14), dtype=object)

    for index, row in x_train_ecg.iterrows():

        new_feat = extrack_features(row,index)
        features = np.vstack([features,  new_feat])
    return pd.DataFrame(features)
#classify_ecg(x_train)

In [62]:

#x_train = x_train.iloc[np.argwhere(labels_array == 1).flatten()]

In [63]:
x_train = classify_ecg(x_train)

C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

In [64]:
id_error

[]

In [65]:
#scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#x_train = scaler.fit_transform(x_train)
x_test = classify_ecg(x_test)
#x_test = scaler.transform(x_test)
x_train.to_csv("x_train_preprocessed.csv",index_label="id")
x_test.to_csv("x_test_preprocessed.csv",index_label="id")

C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

'[8575] not in index'  -  1247  error


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

'[7565] not in index'  -  4203  error


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

'[8073] not in index'  -  962  error


C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _metho

In [66]:
from sklearn import svm

class_weights = {}
for c in range(4):
    class_weights.update({c:len(y_train) / float(np.count_nonzero(y_train == c))})


svm_model = svm.SVC( kernel='rbf', degree=3, gamma='auto',
                coef0=0.0, shrinking=True, tol=0.001,
                cache_size=200, class_weight=class_weights, verbose=False,
                max_iter=-1, decision_function_shape='ovo', random_state=None)

svm_model.fit(x_train, y_train)

C:\Users\mmazz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(class_weight={0: 1.6934215970211006, 1: 11.213698630136987,
                  2: 3.4804421768707483, 3: 30.31851851851852},
    decision_function_shape='ovo', gamma='auto')

In [67]:
#Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
mod = GradientBoostingClassifier(n_estimators=100, random_state=0).fit(x_train, np.ravel(y_train))


In [68]:
y_pred_gbc = mod.predict(np.array(x_test))

In [69]:
from sklearn.metrics import f1_score
def performance(y_pred):
    varb = y_test.empty
    if not y_test.empty:
        return f1_score(y_test, y_pred, average='macro')

In [70]:

y_pred_svc = svm_model.predict(np.array(x_test))
performance(y_pred_gbc)

0.4641154349414452

In [71]:
performance(y_pred_svc)


0.18882262996941895

In [ ]:
x_train.shape


(4093, 14)